In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_columns', None)

# 📊 Data Process

Bu bölümde, günlük olarak kaydedilen Excel dosyaları sırasıyla okunmuş ve tek bir aylık veri setine dönüştürülmüştür. İşlem adımları şu şekildedir:

1. **Günlük Excel dosyaları**, `os.listdir()` kullanılarak sıralı bir şekilde okundu.
2. **Tüm dosyalar birleştirilerek** tek bir DataFrame elde edildi.
3. **Veri temizliği** adımında hatalı, eksik veya yinelenen satırlar çıkarıldı.
4. Ham veri satır sayıs652*065250,903**  
   Temizlenmiş nihai veri se651,975,250,816** satır

> ℹ️ Not: Veri işleme sırasında tarih biçimleri dönüştürülmüş ve gereksiz sütunlar kaldırılmıştır.
dırıldı.


In [2]:
import pandas as pd

# Kolon başlıkları
columns = [
    "ARAC_NO", "ISLEM_TIPI", "BILET_SERI_NO", "ENLEM", "BOYLAM",
    "TIMESTAMP", "HAT_NO", "ALT_HAT_NO", "BILET_ADI"
]

# Dosya yolu formatı
base_path = 'C:\\Users\\Paraboly\\Desktop\\Projeler\\Konya\\Veriler\\nisan2024_konya_smart_card\\SmartCard_202404{:02d}.csv'

# Tüm günleri kapsayacak birleştirilmiş dataframe
all_data = pd.DataFrame()

# ID'yi takip eden sayaç
current_id = 0

# 7'den 13'e kadar olan günler
for day in range(1, 31):
    file_path = base_path.format(day)
    
    df = pd.read_csv(
        file_path,
        sep=";",
        encoding='ISO-8859-9',
        names=columns,
        skiprows=1
    )
    
    # ID ekle ve güncelle
    df['ID'] = range(current_id, current_id + len(df))
    current_id += len(df)
    
    # Kolon sırasını güncelle
    df = df[['ID'] + columns]
    
    # DataFrame’i birleştir
    all_data = pd.concat([all_data, df], ignore_index=True)


C:\Users\Paraboly\AppData\Local\Temp\ipykernel_19108\3473252795.py:22: DtypeWarning: Columns (0,3,4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
C:\Users\Paraboly\AppData\Local\Temp\ipykernel_19108\3473252795.py:22: DtypeWarning: Columns (0,3,4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
C:\Users\Paraboly\AppData\Local\Temp\ipykernel_19108\3473252795.py:22: DtypeWarning: Columns (0,3,4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
C:\Users\Paraboly\AppData\Local\Temp\ipykernel_19108\3473252795.py:22: DtypeWarning: Columns (0,3,4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
C:\Users\Paraboly\AppData\Local\Temp\ipykernel_19108\3473252795.py:22: DtypeWarning: Columns (0,3,4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
C:\Us

In [3]:
len(all_data)

8652065

In [4]:
# Silinmesi gereken satırı temsil eden liste (başlık satırı gibi görünüyor)
# 1. Başlık satırı gibi olan satırları sil
all_data = all_data[~(
    (all_data['ARAC_NO'] == 'ARAC_NO') &
    (all_data['ISLEM_TIPI'] == 'ISLEM_TIPI') &
    (all_data['BILET_SERI_NO'] == 'BILET_SERI_NO') &
    (all_data['ENLEM'] == 'ENLEM') &
    (all_data['BOYLAM'] == 'BOYLAM') &
    (all_data['TIMESTAMP'] == 'TIMESTAMP') &
    (all_data['HAT_NO'] == 'HAT_NO') &
    (all_data['ALT_HAT_NO'] == 'ALT_HAT_NO') &
    (all_data['BILET_ADI'] == 'BILET_ADI')
)]

# 2. "-------" gibi ayraç içeren satırları sil
def is_dash_row(row):
    return all(str(val).strip().startswith('-') for val in row[1:])  # ID'yi atlıyoruz

all_data = all_data[~all_data.apply(is_dash_row, axis=1)]

all_data = all_data.dropna(subset=['ISLEM_TIPI'])
len(all_data)

8651975

### 📈 Figür 1: Aylık İşlem Tipi Dağılımı

Aşağıdaki grafik, tüm kayıtlar içerisinde işlem tiplerinin yüzdesel dağılımını göstermektedir.

In [5]:
import pandas as pd

# İşlem tipi sayımı ve yüzdelik hesaplama
islem_tipi_counts = all_data['ISLEM_TIPI'].value_counts(dropna=False)
islem_tipi_percent = all_data['ISLEM_TIPI'].value_counts(normalize=True, dropna=False) * 100

# DataFrame'e dönüştürme
islem_tipi_df = pd.DataFrame({
    'İşlem Tipi': islem_tipi_counts.index,
    'Toplam': islem_tipi_counts.values,
    'Yüzde (%)': islem_tipi_percent.round(1).values
})

# Gerekirse index sıfırlanır
islem_tipi_df = islem_tipi_df.reset_index(drop=True)
islem_tipi_df.head(2)

,İşlem Tipi,Toplam,Yüzde (%)
0,BILET,6155750,71.1
1,ABILET,1357756,15.7


In [7]:
output_path = r"C:\Users\Paraboly\Desktop\Projeler\Konya\Analiz_Export\SmartCard_Analizleri\Exceller\Figure_1_islem_tipi_dagilimi.xlsx"

# Excel'e export et
islem_tipi_df.to_excel(output_path, index=False)


### 📈 Figür 2: Aylık İşlem Tipi Dağılımı

Aşağıdaki grafik, tüm kayıtlar içerisinde işlem tiplerinin yüzdesel dağılımını göstermektedir.

In [8]:
import pandas as pd

# İşlem tipi sayımı ve yüzdelik hesaplama
islem_tipi_counts = all_data['BILET_ADI'].value_counts(dropna=False)
islem_tipi_percent = all_data['BILET_ADI'].value_counts(normalize=True, dropna=False) * 100

# DataFrame'e dönüştürme
islem_tipi_df = pd.DataFrame({
    'BILET_ADI': islem_tipi_counts.index,
    'Toplam': islem_tipi_counts.values,
    'Yüzde (%)': islem_tipi_percent.round(1).values
})

# Gerekirse index sıfırlanır
islem_tipi_df = islem_tipi_df.reset_index(drop=True)
islem_tipi_df.head(2)

,BILET_ADI,Toplam,Yüzde (%)
0,İNDİRİMLİ,4238831,49.0
1,TAM KART,2658829,30.7


In [9]:
output_path = r"C:\Users\Paraboly\Desktop\Projeler\Konya\Analiz_Export\SmartCard_Analizleri\Exceller\Figure_2_bilet_tipi_dagilimi.xlsx"

# Excel'e export et
islem_tipi_df.to_excel(output_path, index=False)


### 📈 Figür 3: Aylık Hat No Dağılımı

Aşağıdaki grafik, tüm kayıtlar içerisinde işlem tiplerinin yüzdesel dağılımını göstermektedir.

In [10]:
# Sayıya çevrilebilen HAT_NO'ları filtrele
all_data_numeric = all_data[pd.to_numeric(all_data['HAT_NO'], errors='coerce').notna()]

# HAT_NO'yu sayıya çevir (tamsayı)
all_data_numeric['HAT_NO'] = all_data_numeric['HAT_NO'].astype(int)

# HAT_NO'ya göre sayım
hat_no_counts = all_data_numeric['HAT_NO'].value_counts().sort_index()

# DataFrame'e çevir
hat_no_df = hat_no_counts.reset_index()
hat_no_df.columns = ['HAT NO', 'Biniş Sayısı']

# Genel toplamı ekle
total_row = pd.DataFrame([{'HAT NO': 'Genel Toplam', 'Biniş Sayısı': hat_no_df['Biniş Sayısı'].sum()}])
hat_no_df = pd.concat([hat_no_df, total_row], ignore_index=True)
# 'Genel Toplam' satırını dışarıda bırak
top10_hat = hat_no_df[hat_no_df['HAT NO'] != 'Genel Toplam'].sort_values(by='Biniş Sayısı', ascending=False).head(10)

top10_hat 


,HAT NO,Biniş Sayısı
117,150,2463596
114,140,458380
61,65,353562
78,84,237703
118,151,204083
115,141,187154
49,52,172176
39,41,159058
30,31,142269
13,14,131428


In [11]:
output_path = r"C:\Users\Paraboly\Desktop\Projeler\Konya\Analiz_Export\SmartCard_Analizleri\Exceller\Figure_3_top_10_binis_sayısı.xlsx"

# Excel'e export et
top10_hat.to_excel(output_path, index=False)


### 📈 Figür 4: Günlük Biniş Dağılımı


In [20]:
# TIMESTAMP kolonunu datetime formatına çevir
all_data['TIMESTAMP'] = pd.to_datetime(all_data['TIMESTAMP'])

# Tarih kolonunu oluştur
all_data['TARIH'] = all_data['TIMESTAMP'].dt.date

# Günlük biniş sayısı
daily_counts = all_data['TARIH'].value_counts().sort_index()
daily_counts_df = daily_counts.reset_index()
daily_counts_df.columns = ['TARIH', 'BINIŞ_SAYISI']

# Genel toplam satırı
total_row = pd.DataFrame([{
    'TARIH': 'Genel Toplam',
    'BINIŞ_SAYISI': daily_counts_df['BINIŞ_SAYISI'].sum()
}])

# Toplamı ekle
daily_counts_df = pd.concat([daily_counts_df, total_row], ignore_index=True)

daily_counts_df


,TARIH,BINIŞ_SAYISI
0,2024-04-01,309018
1,2024-04-02,392846
2,2024-04-03,390792
3,2024-04-04,391200
4,2024-04-05,387114
5,2024-04-06,266542
6,2024-04-07,185064
7,2024-04-08,214490
8,2024-04-09,165542
9,2024-04-10,1740


In [22]:
output_path = r"C:\Users\Paraboly\Desktop\Projeler\Konya\Analiz_Export\SmartCard_Analizleri\Exceller\Figure_4_Günlük_Binis_dağılımı.xlsx"

# Excel'e export et
daily_counts_df.to_excel(output_path, index=False)


### 📈 Figür 5: Ortalama Saatlik Biniş Dağılımı


In [14]:
# TIMESTAMP'i datetime formatına çevir (zaten yaptıysan gerek yok)
all_data['TIMESTAMP'] = pd.to_datetime(all_data['TIMESTAMP'])

# Tarih ve saat kolonlarını oluştur
all_data['TARIH'] = all_data['TIMESTAMP'].dt.date
all_data['SAAT'] = all_data['TIMESTAMP'].dt.hour

# Tarih ve saat bazında gruplama: her gün her saat için biniş sayısı
daily_hourly = all_data.groupby(['TARIH', 'SAAT']).size().reset_index(name='BINIŞ_SAYISI')

# Şimdi saat bazında ortalama alalım
hourly_avg = daily_hourly.groupby('SAAT')['BINIŞ_SAYISI'].mean().round(2)

# Etiketleri oluştur: 00:00-01:00, 01:00-02:00, ...
labels = [f"{str(h).zfill(2)}:00-{str((h+1)%24).zfill(2)}:00" for h in hourly_avg.index]

# DataFrame'e dönüştür
hourly_avg_df = pd.DataFrame({
    'ZAMAN_ARALIĞI': labels,
    'ORTALAMA_BINIŞ_SAYISI': hourly_avg.values
})

# Genel ortalama (24 saatin ortalaması)
genel_ortalama_row = pd.DataFrame([{
    'ZAMAN_ARALIĞI': 'Genel Ortalama',
    'ORTALAMA_BINIŞ_SAYISI': hourly_avg_df['ORTALAMA_BINIŞ_SAYISI'].mean().round(1)
}])

# Ekle
hourly_avg_df = pd.concat([hourly_avg_df, genel_ortalama_row], ignore_index=True)

# Sonuç
hourly_avg_df.head(5)


,ZAMAN_ARALIĞI,ORTALAMA_BINIŞ_SAYISI
0,00:00-01:00,976.87
1,01:00-02:00,75.52
2,02:00-03:00,8.57
3,03:00-04:00,34.32
4,04:00-05:00,13.78


In [15]:
output_path = r"C:\Users\Paraboly\Desktop\Projeler\Konya\Analiz_Export\SmartCard_Analizleri\Exceller\Figure_5_Ortalama_Saatlik_Binis_Dagılımı.xlsx"

# Excel'e export et
hourly_avg_df.to_excel(output_path, index=False)


### 📈 Figür 6: Hangi saatlerde hangi bilet türü daha çok kullanılıyor? (ör: öğrenci sabah yoğunluğu)


In [16]:
import pandas as pd

# Öncelikle filtrelenecek bilet türleri
selected_bilets = [
    "65 YAŞ ÜSTÜ",
    "DEPREMZEDE SERBEST",
    "ENGELLİ",
    "ENGELLİ B",
    "GÖRME ENGELLİ",
    "KAMU TAM ABONMAN",
    "TAM KART",
    "İNDİRİMLİ"
]

# Seçilen bilet türlerine göre filtrele
filtered_data = all_data[all_data['BILET_ADI'].isin(selected_bilets)].copy()

# ENGELLİ türlerini birleştir
filtered_data['BILET_ADI'] = filtered_data['BILET_ADI'].replace({
    "ENGELLİ": "ENGELLİ",
    "ENGELLİ B": "ENGELLİ",
    "GÖRME ENGELLİ": "ENGELLİ"
})

# TIMESTAMP datetime formatında değilse çevir
filtered_data['TIMESTAMP'] = pd.to_datetime(filtered_data['TIMESTAMP'])

# Tarih ve saat sütunlarını oluştur
filtered_data['TARIH'] = filtered_data['TIMESTAMP'].dt.date
filtered_data['SAAT'] = filtered_data['TIMESTAMP'].dt.hour

# Her gün, saat ve bilet türü için biniş sayısı
daily_hourly_bilet = filtered_data.groupby(['TARIH', 'SAAT', 'BILET_ADI']).size().reset_index(name='BINIŞ_SAYISI')

# Saat ve bilet türü bazında günlük ortalama biniş sayısı
hourly_avg = daily_hourly_bilet.groupby(['SAAT', 'BILET_ADI'])['BINIŞ_SAYISI'].mean().unstack(fill_value=0).round(2)

# İstersen index olan saatleri etiket olarak değiştirebilirsin
hour_labels = [f"{str(h).zfill(2)}:00-{str((h+1)%24).zfill(2)}:00" for h in hourly_avg.index]
hourly_avg.index = hour_labels

bilet_totals = hourly_avg.sum(axis=0)

# Yüzdesel dağılımı hesapla
hourly_percent = hourly_avg.divide(bilet_totals, axis=1) * 100

# Ondalık kısımları 2 basamakla sınırla
hourly_percent = hourly_percent.round(1)
hourly_percent_reset = hourly_percent.reset_index(names='BILET_ADI')
hourly_percent_reset

BILET_ADI,BILET_ADI,65 YAŞ ÜSTÜ,DEPREMZEDE SERBEST,ENGELLİ,TAM KART,İNDİRİMLİ
0,00:00-01:00,0.1,0.3,0.3,0.5,0.3
1,01:00-02:00,0.0,0.1,0.0,0.0,0.0
2,02:00-03:00,0.0,0.0,0.0,0.0,0.0
3,03:00-04:00,0.0,0.0,0.0,0.0,0.0
4,04:00-05:00,0.0,0.0,0.0,0.0,0.0
5,05:00-06:00,0.0,0.1,0.1,0.1,0.0
6,06:00-07:00,1.0,1.1,4.0,2.9,1.6
7,07:00-08:00,2.3,5.5,6.9,7.4,9.4
8,08:00-09:00,4.2,7.8,5.6,6.5,10.0
9,09:00-10:00,6.9,5.2,5.0,4.8,4.5


In [17]:
output_path = r"C:\Users\Paraboly\Desktop\Projeler\Konya\Analiz_Export\SmartCard_Analizleri\Exceller\Figure_6_Saatlik_Bilet_Turu_Yuzdesel_Dagilim.xlsx"

# Excel'e export et
hourly_percent_reset.to_excel(output_path, index=False)


### 📈 Figür 7:Hatların saatlik kullanım desenleri


In [28]:
import pandas as pd

# Seçmek istediğin hat numaraları
selected_hats = [150, 140, 65, 84, 151, 141, 52, 41, 31, 14]

# HAT_NO ve SAAT sütunlarının uygun tipte olduğundan emin ol
all_data['HAT_NO'] = pd.to_numeric(all_data['HAT_NO'], errors='coerce')
all_data['SAAT'] = pd.to_numeric(all_data['SAAT'], errors='coerce')
all_data = all_data.dropna(subset=['HAT_NO', 'SAAT', 'TIMESTAMP'])

# Tür dönüşümleri
all_data['HAT_NO'] = all_data['HAT_NO'].astype(int)
all_data['SAAT'] = all_data['SAAT'].astype(int)

# Sadece seçilen hatları filtrele
filtered_hats = all_data[all_data['HAT_NO'].isin(selected_hats)].copy()

# TIMESTAMP datetime formatında değilse çevir ve TARIH sütunu oluştur
filtered_hats['TIMESTAMP'] = pd.to_datetime(filtered_hats['TIMESTAMP'])
filtered_hats['TARIH'] = filtered_hats['TIMESTAMP'].dt.date

# Gün, saat ve hat bazında biniş sayısı
daily_hat_hour_counts = filtered_hats.groupby(['TARIH', 'HAT_NO', 'SAAT']).size().reset_index(name='BINIŞ_SAYISI')

# Saat ve hat bazında günlük ortalama biniş sayısı
hourly_avg = daily_hat_hour_counts.groupby(['HAT_NO', 'SAAT'])['BINIŞ_SAYISI'].mean().reset_index()

# Pivot tabloya çevir (satır: HAT_NO, sütun: SAAT)
hourly_avg_pivot = hourly_avg.pivot(index='HAT_NO', columns='SAAT', values='BINIŞ_SAYISI').fillna(0)

# Veriyi yuvarla
hourly_avg_pivot = hourly_avg_pivot.round(1)

# Index'i sıfırla
hourly_avg_pivot_reset = hourly_avg_pivot.reset_index()

# Saat aralığı etiketlerini oluştur (0–23 saatleri için)
hour_labels = [f"{str(h).zfill(2)}:00-{str((h + 1) % 24).zfill(2)}:00" for h in range(24)]

# Yeni sütun adlarını ata
hourly_avg_pivot_reset.columns = ['HAT_NO'] + hour_labels

hourly_avg_pivot_reset


,HAT_NO,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,06:00-07:00,07:00-08:00,08:00-09:00,09:00-10:00,10:00-11:00,11:00-12:00,12:00-13:00,13:00-14:00,14:00-15:00,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00
0,14,7.2,0.0,0.0,0.0,0.0,1.1,78.6,611.2,526.6,220.1,179.9,218.2,254.2,294.5,310.5,522.6,461.2,399.4,317.9,145.0,101.9,58.8,42.9,38.7
1,31,30.6,2.0,0.0,0.0,0.0,2.0,290.0,549.4,368.5,211.4,218.2,268.3,294.2,341.8,348.2,480.7,369.5,352.7,329.2,215.1,178.2,107.5,91.9,59.8
2,41,31.8,7.0,0.0,0.0,1.0,0.0,109.7,457.9,421.7,323.6,258.8,267.5,394.9,450.1,476.3,567.9,559.4,462.2,372.5,207.4,172.2,117.1,92.1,79.5
3,52,11.1,0.0,0.0,0.0,1.0,0.0,71.8,449.8,545.4,362.4,306.7,346.1,426.8,492.6,465.3,621.4,573.7,455.5,306.0,185.4,181.1,115.4,100.4,84.4
4,65,68.6,3.7,0.0,0.0,1.0,1.0,313.8,951.2,1030.4,615.5,564.6,732.9,902.2,1014.1,938.2,1311.8,1058.9,893.9,646.1,402.6,418.6,294.9,256.8,177.0
5,84,46.2,0.0,0.0,0.0,1.0,20.3,468.3,723.1,606.5,378.4,393.0,459.2,545.0,562.7,540.4,748.3,703.5,644.6,510.9,367.1,319.5,202.2,192.1,127.9
6,140,133.5,1.4,0.0,0.0,0.0,1.5,118.1,771.0,837.9,609.1,571.3,651.6,855.9,983.3,1005.9,1256.5,1693.3,1803.3,1656.4,1021.6,928.0,756.7,726.4,487.7
7,141,34.1,1.5,0.0,2.0,0.0,1.0,103.7,458.6,419.2,281.7,246.9,295.3,392.5,417.4,459.8,579.7,639.5,697.2,645.1,399.8,279.8,219.9,221.3,150.3
8,150,216.6,10.4,2.7,5.2,3.2,66.6,1048.3,5369.0,5992.1,4033.0,3573.0,4200.5,5307.6,5399.0,5949.3,6696.0,7563.8,6932.7,5905.0,3664.6,3418.5,2703.6,2719.0,1417.8
9,151,13.1,92.0,14.0,3.0,0.0,1.4,85.0,376.6,378.3,300.0,319.9,420.7,576.0,593.1,651.8,920.5,805.3,610.2,436.9,224.3,214.3,143.3,132.2,77.8


In [30]:
output_path = r"C:\Users\Paraboly\Desktop\Projeler\Konya\Analiz_Export\SmartCard_Analizleri\Exceller\Figure_7_hat_bazli_saatlik_binis_oranlari.xlsx"

# Excel'e export et
hourly_avg_pivot_reset.to_excel(output_path, index=False)


### 📈 Figür 8:Transfer Noktaları ve BSCAN (yoğunluk tabanlı kümelenme)


In [ ]:
import pandas as pd
all_data = all_data[(all_data['ENLEM'] != 0) & (all_data['BOYLAM'] != 0)]

# TIMESTAMP'i datetime formatına çevir
all_data['TIMESTAMP'] = pd.to_datetime(all_data['TIMESTAMP'])

# Gerekli sütunları al
df = all_data[['BILET_SERI_NO', 'TIMESTAMP', 'HAT_NO', 'ENLEM', 'BOYLAM']].copy()

# HAT_NO sayıya çevrilmeli (bazı hatalar olabilir)
df['HAT_NO'] = pd.to_numeric(df['HAT_NO'], errors='coerce').astype('Int64')

# Veriyi sırala
df = df.sort_values(by=['BILET_SERI_NO', 'TIMESTAMP'])

# Şimdi her kişi (BILET_SERI_NO) için birden fazla transferi gözlemleyeceğiz
# Bunu yapmak için bir önceki kayıtla fark alacağız (shift ile değil, df.groupby içinde ardışık kayıtları kontrol ederek)

transfer_list = []

for ticket, group in df.groupby('BILET_SERI_NO'):
    group = group.reset_index(drop=True)
    for i in range(len(group) - 1):
        current = group.loc[i]
        for j in range(i+1, len(group)):
            next_row = group.loc[j]
            time_diff = (next_row['TIMESTAMP'] - current['TIMESTAMP']).total_seconds() / 60

            # 90 dakikayı geçtiyse artık aramayı bırak
            if time_diff > 90:
                break

            # HAT değişmişse geçerli transfer
            if current['HAT_NO'] != next_row['HAT_NO']:
                transfer_list.append({
                    'BILET_SERI_NO': ticket,
                    'FROM_HAT': current['HAT_NO'],
                    'TO_HAT': next_row['HAT_NO'],
                    'TRANSFER_TIME': next_row['TIMESTAMP'],
                    'ENLEM': next_row['ENLEM'],
                    'BOYLAM': next_row['BOYLAM']
                })
                break  # sadece ilk geçerli transferi al, istersen burayı kaldırıp tüm geçerli olanları alabilirsin

# Transferleri DataFrame'e çevir
transfer_df = pd.DataFrame(transfer_list)

# Transfer noktalarını al
transfer_points = transfer_df[['TO_HAT', 'ENLEM', 'BOYLAM']].drop_duplicates()
transfer_points.columns = ['HAT_NO', 'ENLEM', 'BOYLAM']


In [22]:
transfer_points

,HAT_NO,ENLEM,BOYLAM
0,151,37.8687867,32.5417533
1,150,0.0000000,0.0000000
2,61,37.976888,32.515652
3,150,37.9765383,32.5161383
4,61,37.97681,32.515513
...,...,...,...
1023891,86,37.881528,32.490232
1023893,150,37.8718933,32.4902333
1023894,63,37.881512,32.490355
1023898,8,37.87252,32.485212


In [23]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN

# Dünya yarıçapı (metre)
EARTH_RADIUS = 6371000

# Metreyi radyana çeviren fonksiyon
def meters_to_radians(meters):
    return meters / EARTH_RADIUS

# eps (10 metre) → radyan
eps_rad = meters_to_radians(10)

# ENLEM ve BOYLAM'ı sayısala çevir
transfer_points['ENLEM'] = pd.to_numeric(transfer_points['ENLEM'], errors='coerce')
transfer_points['BOYLAM'] = pd.to_numeric(transfer_points['BOYLAM'], errors='coerce')

# Geçerli koordinatları filtrele
transfer_points_cleaned = transfer_points[
    (transfer_points['ENLEM'] > 0) &
    (transfer_points['BOYLAM'] > 0)
].copy()

# Koordinatları NumPy dizisine çevir ve radyana dönüştür
coords = transfer_points_cleaned[['ENLEM', 'BOYLAM']].to_numpy()
coords_rad = np.radians(coords)

# DBSCAN ile kümelendirme
db = DBSCAN(eps=eps_rad, min_samples=1, algorithm='ball_tree', metric='haversine')
labels = db.fit_predict(coords_rad)

# Kümelere ait etiketleri ekle
transfer_points_cleaned['CLUSTER_ID'] = labels

# Küme merkezlerini hesapla
centroids = (
    transfer_points_cleaned.groupby('CLUSTER_ID')[['ENLEM', 'BOYLAM']]
    .mean()
    .rename(columns={'ENLEM': 'CENTER_ENLEM', 'BOYLAM': 'CENTER_BOYLAM'})
)

# Merkez bilgilerini ana tabloya ekle (merge daha güvenlidir)
transfer_points_clustered = pd.merge(
    transfer_points_cleaned,
    centroids,
    left_on='CLUSTER_ID',
    right_index=True,
    how='left'
)

# Sonuç: transfer_points_clustered


In [26]:
transfer_points_clustered

,HAT_NO,ENLEM,BOYLAM,CLUSTER_ID,CENTER_ENLEM,CENTER_BOYLAM
0,151,37.868787,32.541753,0,37.868817,32.542126
2,61,37.976888,32.515652,1,37.976682,32.515919
3,150,37.976538,32.516138,1,37.976682,32.515919
4,61,37.976810,32.515513,1,37.976682,32.515919
5,150,37.976565,32.516087,1,37.976682,32.515919
...,...,...,...,...,...,...
1023891,86,37.881528,32.490232,56,37.881590,32.489764
1023893,150,37.871893,32.490233,3,37.872152,32.489875
1023894,63,37.881512,32.490355,56,37.881590,32.489764
1023898,8,37.872520,32.485212,29,37.872521,32.485207


In [40]:
grouped = (
    transfer_points_clustered
    .groupby(['CENTER_ENLEM', 'CENTER_BOYLAM'])
    .size()
    .reset_index(name='COUNT')
)

# COUNT > 1 olanları filtrele
grouped = grouped[grouped['COUNT'] > 10]

# Büyükten küçüğe sırala
grouped = grouped.sort_values(by='COUNT', ascending=False).reset_index(drop=True)
grouped

,CENTER_ENLEM,CENTER_BOYLAM,COUNT
0,37.872152,32.489875,102466
1,37.873370,32.496159,49209
2,37.868443,32.485631,35724
3,37.899848,32.488893,30018
4,37.976682,32.515919,29289
...,...,...,...
1094,37.862313,32.424663,11
1095,37.905972,32.481918,11
1096,37.974677,32.515297,11
1097,37.906568,32.467150,11


In [43]:
import folium
from folium.plugins import HeatMap
import pandas as pd
import branca

# Örnek grouped verisi (senin verinle değiştirilecek)
# grouped = ...

# Harita merkezi
map_center = [grouped['CENTER_ENLEM'].mean(), grouped['CENTER_BOYLAM'].mean()]
m = folium.Map(location=map_center, zoom_start=13)

# HeatMap verisi
heat_data = [
    [row['CENTER_ENLEM'], row['CENTER_BOYLAM'], row['COUNT']]
    for _, row in grouped.iterrows()
]

# Isı haritasını ekle
HeatMap(
    heat_data,
    min_opacity=0.5,
    radius=15,
    blur=5,
    max_zoom=18
).add_to(m)

# 🔥 Renk skalası için özel HTML legend
legend_html ='''
<div style="
     position: fixed;
     bottom: 40px;
     left: 40px;
     width: 200px;
     height: auto;
     background-color: white;
     z-index:9999;
     font-size:14px;
     border: 2px solid grey;
     border-radius: 8px;
     box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
     padding: 12px;
">
<b>Transfer Yoğunluğu</b><br><br>
<div style="line-height: 1.6;">
  <span style="background-color:blue; width:20px; height:20px; display:inline-block; border:1px solid #000;"></span>
  &nbsp; Düşük<br>
  <span style="background-color:lime; width:20px; height:20px; display:inline-block; border:1px solid #000;"></span>
  &nbsp; Orta<br>
  <span style="background-color:red; width:20px; height:20px; display:inline-block; border:1px solid #000;"></span>
  &nbsp; Yüksek
</div>
</div>
'''

m.get_root().html.add_child(folium.Element(legend_html))

# Göster
m


In [ ]:
grouped

In [44]:
output_path = r"C:\Users\Paraboly\Desktop\Projeler\Konya\Analiz_Export\SmartCard_Analizleri\Exceller\transfer_lokasyonları.xlsx"

# Excel'e export et
grouped.to_excel(output_path, index=False)
